# Pickle

Umožňuje binárně "zmrazit" prakticky libovolná data.

In [ ]:
import pickle

In [ ]:
# struktury
data = {
    "velmi": "slozity",
    "slovnik": ["slozeny", "z", 1.2, "klicu"],
    "a": {"dalsich", "divnych", "veci"},
    (1, 2): False,
    5: None,
}

In [ ]:
# zápis do souboru
with open("data/pickle.p", mode="wb") as f:
    pickle.dump(data, f)

In [ ]:
# zapiklení do objektu `bytes`
pickle.dumps(data)[:64]  # zkráceno ..

In [ ]:
# načtení ze souboru
with open("data/pickle.p", mode="rb") as f:
    print(pickle.load(f))

**Co lze piklit?**

- `None`, `True`, `False`
- celá, reálná a komplexní čísla
- řetězce, `bytes` a `bytearray`
- n-tice, seznamy, množiny a slovníky s piklitelnými daty
- běžné funkce (ne uzávěry, ne lambdy)
- třídy (ne vnořené)
- instance tříd, jejichž atribut `__dict__` nebo návratová hodnota metody `__getstate__()` je piklitelná

U tříd a funkcí se piklí jenom *plné jméno objektu, ne kód*!   U instancí navíc jejich data.

Co to znamená?

In [ ]:
def udelej_vec():
    print("Dělám věc.")

zapiklena_funkce = pickle.dumps(udelej_vec)

# -------- přenos dat --------

def udelej_vec():
    print("Dělám VELMI ZLOU věc.")

odpiklena_funkce = pickle.loads(zapiklena_funkce)
odpiklena_funkce()

In [ ]:
# chyby
try:
    pickle.dumps(lambda x: x ** 2)

except pickle.PickleError as e:
    print(e)

except RecursionError:
    print("We cannot go deeper.")

**Nikdy nerozpiklujte cizí data.**

# Shelve

= Automaticky piklící slovník.

In [ ]:
import shelve

In [ ]:
# zápis
with shelve.open("data/storage") as storage:
    storage["description"] = "dangerous content"
    storage["function"] = abs

In [ ]:
# čtení = ten samý protokol! :-)
with shelve.open("data/storage") as storage:
    print(dict(storage))

**Nikdy nerozpiklujte cizí data.**

# [PEP249](https://www.python.org/dev/peps/pep-0249/) – Python Database API Specification 2.0

Každý DB modul by měl splňovat:

- `modul.connect()` – vytvoření spojení
- `connection.cursor()` – vytvoření kurzoru
- `cursor.execute()` – vykonání dotazu
- `cursor.fetchall()` – vytažení výsledků dotazu
- `cursor.rowcount` – počet ovlivněných (vybraných) řádků dotazem
- `connection.close()` – uzavření spojení

# SQLite3

= Nenáročná, relativně hloupá DB.

In [ ]:
import sqlite3  # splňuje PEP249 - DB-API 2.0

In [ ]:
# vytvoření spojení a kurzoru
connection = sqlite3.connect(":memory:")
c = connection.cursor()

In [ ]:
# vytvoření tabulky DB
c.execute("""
CREATE TABLE IF NOT EXISTS samples
(id INTEGER PRIMARY KEY AUTOINCREMENT, value REAL);
""")

In [ ]:
from random import random

# naplnění
for i in range(10):
    params = (random(), ) # tuple
    c.execute("INSERT INTO samples (value) VALUES (?)",  params)

# potvrzení transakce
connection.commit()

In [ ]:
# výběr
c.execute("SELECT * FROM samples")

# SQLite3 implementuje metodu `__iter__` u kurzoru, jinak => `fetchall`
for sample in c:
    print(sample)

In [ ]:
# jiný obal pro vracená data
connection.row_factory = sqlite3.Row

# musí být před vytvořením kurzoru
advanced_cursor = connection.cursor()

advanced_cursor.execute("SELECT * FROM samples")
for sample in advanced_cursor:
    print(dict(sample))

In [ ]:
# kontextový manažer vytvoří transakci
with connection:
    # `connection.execute` = zkratka bez vytváření kurzoru
    connection.execute("INSERT INTO samples (value) VALUES (0)")  # nebude vloženo
    connection.execute("INVALID QUERY")

# ORM

= Objektově relační mapování. Není ve standardní knihovně.

- [SQLAlchemy](http://www.sqlalchemy.org/)
- [peewee](http://peewee.readthedocs.io/en/latest/index.html)

## Peewee

In [ ]:
import peewee
from datetime import date

In [ ]:
db = peewee.SqliteDatabase("data/people.db")

# společný předek pro vlastní modely
class Model(peewee.Model):
    class Meta:
        database = db

# vlastní model
class Person(Model):
    name = peewee.CharField()
    birthday = peewee.DateField()

In [ ]:
db.connect()

In [ ]:
# toto se běžně nedělá
if Person.table_exists():
    Person.drop_table()
Person.create_table()

In [ ]:
# vytvoření záznamů
Person.create(name="David", birthday=date.today())
luke = Person(name="Luke", birthday=date(2002, 1, 3))
luke.save()

In [ ]:
# výběr
for person in Person.select():
    print(person.name, person.birthday)

In [ ]:
# aktualizace
luke.birthday = date.today()
luke.save()

In [ ]:
# filtrovaný výběr
for person in Person.select().where(Person.name == "Luke"):
    print(person.birthday)

**Podporované backendy:**

- SQLite (pomocí standardní knihovny)
- MySQL – `pymysql`
- PostgreQSL - `psycopg2`

# Příklad

- Přečtěte soubor `data/books.xml` a data z něj vložte do SQLite databáze `data/books.sqlite3`.
- Vypište knihy, kde: `price > 10.0`.
- *Profi varianta:* nainstalujte a použijte ORM.

In [ ]:
import sqlite3

create_query = r"""
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY,
    title TEXT,
    author TEXT,
    genre TEXT,
    price FLOAT,
    publish_date DATE,
    description TEXT
);
"""

# TODO